In [2]:
import pandas as pd
import plotly.graph_objects as go

# Load the dataset
cards_df = pd.read_csv("../../data/processed/analise_cartoes.csv")

# Define the order of the categories for 'tempo_jogo'
tempo_order = ['00-10', '11-20', '21-30', '31-40', '41-45', '45+', '46-60', '61-70', '71-80', '81-90', '90+']

# Convert 'tempo_jogo' to a categorical variable with the defined order
cards_df['tempo_jogo'] = pd.Categorical(cards_df['tempo_jogo'], categories=tempo_order, ordered=True)

# Filter the dataset for specific teams
times = ["Fortaleza", "Botafogo-RJ", "Corinthians", "Palmeiras", "Flamengo", "Sao Paulo", "Internacional",
         "Atletico-MG", "Cruzeiro", "Gremio", "Vasco", "Bahia", "Fluminense",
         "Athletico-PR", "Vitoria", "Cuiaba", "Atletico-GO", "Criciuma", "Juventude", "Bragantino"]

df = cards_df[cards_df['clube'].isin(times)]

# Create radar plots for each position
positions = df['posicao'].unique()

for position in positions:
    # Filter data for the current position
    position_df = df[df['posicao'] == position]

    # Create a radar plot
    fig = go.Figure()

    for team in position_df['clube'].unique():
        team_df = position_df[position_df['clube'] == team]

        # Ensure the data is ordered by tempo_jogo
        team_df = (
            team_df.groupby('tempo_jogo', observed=True)['Quantidade_Cartoes']
            .sum()
            .reindex(tempo_order, fill_value=0)
        )

        # Debugging: Print the team data
        print(f"Team: {team}, Position: {position}")
        print(team_df)

        # Add trace only if there are non-zero values
        if team_df.sum() > 0:
            fig.add_trace(go.Scatterpolar(
                r=team_df.values,  # Quantidade_Cartoes
                theta=tempo_order,  # Match time
                fill='none',  # No fill
                name=team  # Team name
            ))

    # Update layout for the radar plot
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                title="Quantidade de Cartões",
                range=[0, df['Quantidade_Cartoes'].max()]  # Adjust range based on data
            ),
            angularaxis=dict(
                direction="clockwise",
            )
        ),
        title=f"Radar Plot: Quantidade de Cartões por Tempo para Posição {position}",
        legend=dict(
            title="Clube",
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5
        ),
        height=800,
        width=800
    )


Team: Athletico-PR, Position: Atacante
tempo_jogo
00-10     6
11-20     7
21-30     8
31-40     4
41-45    10
45+       4
46-60    18
61-70    10
71-80    18
81-90    27
90+      13
Name: Quantidade_Cartoes, dtype: int64
Team: Atletico-GO, Position: Atacante
tempo_jogo
00-10     1
11-20     2
21-30     2
31-40    10
41-45     3
45+       4
46-60     6
61-70    15
71-80    14
81-90    19
90+      12
Name: Quantidade_Cartoes, dtype: int64
Team: Atletico-MG, Position: Atacante
tempo_jogo
00-10     1
11-20     4
21-30    10
31-40     9
41-45     6
45+       2
46-60    13
61-70     7
71-80    14
81-90    28
90+      18
Name: Quantidade_Cartoes, dtype: int64
Team: Bahia, Position: Atacante
tempo_jogo
00-10     2
11-20     4
21-30     4
31-40    11
41-45     7
45+       3
46-60    16
61-70    16
71-80    16
81-90    20
90+      15
Name: Quantidade_Cartoes, dtype: int64
Team: Botafogo-RJ, Position: Atacante
tempo_jogo
00-10     3
11-20     8
21-30     7
31-40     8
41-45     6
45+       2
46-6

In [11]:
cards_df.pivot(index=["clube", "posicao"], columns=["tempo_jogo"], values=['Quantidade_Cartoes']).to_csv(
    "../../data/processed/radar_cartoes_por_tempo_e_posicao.csv"
)